In [1]:
import json

import os

import pandas as pd
import numpy as np

In [2]:
LANG_CODES_TO_NAMES = {
    "en": "English",
    "ru": "Russian",
    "ko": "Korean",
    "cmn-Hant": "Traditional Chinese",
    "ja": "Japanese",
    "de": "German",
    "es": "Spanish",
}

In [3]:
# Function to get the script directory
def get_script_dir():
    """Returns the directory where the script is located."""
    dir = os.getcwd()
    if dir.endswith("modTiers"):
        return dir[:-len("modTiers")]
    return dir

# Function to generate file path
def get_file_path(base_dir: str, file: str, lang: str, is_en=False):
    return f"{base_dir}{lang if not is_en else 'en'}/{file}.json"

# Function to load a JSON file
def load_file(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    with open(path, encoding="utf-8") as f:
        return json.load(f)

# Function to load a DataFrame
def load_df(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    return pd.read_json(path, encoding="utf-8")

def from_json(base_dir: str, file: str, lang: str, is_en=False):
    file = load_file(base_dir, file, lang, is_en)
    return pd.DataFrame(file)

In [4]:
cwd = get_script_dir()
base_dir = cwd + "/../tables/"
lang = "en"

In [5]:
base_items = from_json(base_dir, "BaseItemTypes", lang).drop(columns=["_index"])
stats = from_json(base_dir, "Stats", lang).drop(columns=["_index"])
runes = from_json(base_dir, "SoulCores", lang).drop(columns=["_index"])

In [6]:
stats

,Id,IsLocal,IsWeaponLocal,Semantic,Text,IsVirtual,HASH32,BelongsActiveSkills,IsScalable
0,level,False,False,3,Level,False,1043633784,[],True
1,item_drop_slots,False,False,3,Item Drop Slots,False,2994048877,[],True
2,main_hand_weapon_type,False,False,3,Main Hand Weapon Type,True,356082681,[],True
3,off_hand_weapon_type,False,False,3,Off Hand Weapon Type,True,501678916,[],True
4,current_endurance_charges,False,False,3,Endurance Charges,False,2172576702,[],True
...,...,...,...,...,...,...,...,...,...
22595,ailment_threshold_+_from_%_maximum_energy_shield,False,False,1,,False,364763766,[],True
22596,mana_tempest_empowered_spells_split_to_X_addit...,False,False,3,,False,144893695,[],True
22597,virutal_has_an_elemental_ailment,False,False,4,,True,2792765324,[],True
22598,arc_chain_explode_shocked_enemies,False,False,4,,False,2592581221,[],False


In [7]:
def replace_indices_with_ids(index_list, stats_df):
    return [stats_df.loc[i, 'Id'] for i in index_list]

In [8]:
runes['StatsArmour'] = runes['StatsArmour'].apply(lambda lst: replace_indices_with_ids(lst, stats))
runes['StatsMartialWeapon'] = runes['StatsMartialWeapon'].apply(lambda lst: replace_indices_with_ids(lst, stats))
runes

,BaseItemType,StatsMartialWeapon,StatsValuesMartialWeapon,StatsArmour,StatsValuesArmour,Rank,StatsCasterWeapon,StatsValuesCasterWeapon
0,514,"[local_minimum_added_fire_damage, local_maximu...","[4, 6]",[base_fire_damage_resistance_%],[10],0,[18106],[8]
1,504,"[local_minimum_added_fire_damage, local_maximu...","[7, 11]",[base_fire_damage_resistance_%],[12],15,[18106],[10]
2,524,"[local_minimum_added_fire_damage, local_maximu...","[13, 16]",[base_fire_damage_resistance_%],[14],30,[18106],[12]
3,515,"[local_minimum_added_cold_damage, local_maximu...","[3, 5]",[base_cold_damage_resistance_%],[10],0,[18107],[8]
4,505,"[local_minimum_added_cold_damage, local_maximu...","[6, 10]",[base_cold_damage_resistance_%],[12],15,[18107],[10]
...,...,...,...,...,...,...,...,...
67,571,[],[],[],[],0,[],[]
68,572,[],[],[],[],0,[],[]
69,573,[],[],[],[],0,[],[]
70,574,[],[],[],[],0,[],[]


In [9]:
runes_merged = runes.merge(base_items, left_on="BaseItemType", right_index=True, how="left")
runes_merged[["BaseItemType", "StatsMartialWeapon", "StatsValuesMartialWeapon", "StatsArmour", "StatsValuesArmour"]]

,BaseItemType,StatsMartialWeapon,StatsValuesMartialWeapon,StatsArmour,StatsValuesArmour
0,514,"[local_minimum_added_fire_damage, local_maximu...","[4, 6]",[base_fire_damage_resistance_%],[10]
1,504,"[local_minimum_added_fire_damage, local_maximu...","[7, 11]",[base_fire_damage_resistance_%],[12]
2,524,"[local_minimum_added_fire_damage, local_maximu...","[13, 16]",[base_fire_damage_resistance_%],[14]
3,515,"[local_minimum_added_cold_damage, local_maximu...","[3, 5]",[base_cold_damage_resistance_%],[10]
4,505,"[local_minimum_added_cold_damage, local_maximu...","[6, 10]",[base_cold_damage_resistance_%],[12]
...,...,...,...,...,...
67,571,[],[],[],[]
68,572,[],[],[],[]
69,573,[],[],[],[]
70,574,[],[],[],[]


In [10]:
row = runes_merged[runes_merged["Name"] == "Iron Rune"][
        ["StatsMartialWeapon", "StatsValuesMartialWeapon", "StatsArmour", "StatsValuesArmour"]
    ]
row.to_dict('records')[0]

{'StatsMartialWeapon': ['local_physical_damage_+%'],
 'StatsValuesMartialWeapon': [20],
 'StatsArmour': ['local_armour_and_evasion_and_energy_shield_+%'],
 'StatsValuesArmour': [20]}